# GMM 再测试


In [ ]:
# 确保src目录在Python路径中
import os
import sys

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

sys.path.append(os.path.abspath("../"))

# 导入模块
from src.data_utils import (
    extract_seismic_attributes_for_wells,
    extract_uniform_seismic_samples,
    filter_anomalous_attributes,
    filter_outlier_wells,
    filter_seismic_by_wells,
    identify_attributes,
    parse_petrel_file,
    preprocess_features,
)
from src.feature_selection import select_best_features
from src.gmm_clustering import evaluate_gmm_clusters, perform_gmm_clustering
from src.pca_analysis import perform_pca_analysis
from src.visualization import visualize_attribute_map, visualize_gmm_clustering, visualize_pca_clustering

data_dir = "..\\data"
output_dir = "output"
if not os.path.exists(output_dir):
    os.makedirs(output_dir)


# 设置中文字体
plt.rcParams["font.family"] = "SimHei"  # 黑体 SimHei 支持中文
plt.rcParams["axes.unicode_minus"] = False  # 正常显示负号

## 导入地震数据


In [ ]:
data_seismic_attr = parse_petrel_file(os.path.join(data_dir, "H6-2_attr"))

## 导入井点位置


In [ ]:
data_well_position = pd.read_excel(os.path.join(data_dir, "well_without_attr.xlsx"))

# 选择对应层位的行，丢弃砂厚为 NaN 的行
data_well_purpose_surface_position = (
    data_well_position[data_well_position["Surface"] == "H6-2"]
    .replace(-999, np.nan)  # 将-999替换为NaN
    .dropna(subset=["Sand Thickness"])  # 丢弃砂厚为NaN的行
    .reset_index(drop=True)  # 重置索引
)
data_well_purpose_surface_position.head()

## 筛除离群井


In [ ]:
# 筛选离群井
data_well_purpose_surface_filtered = filter_outlier_wells(data_well_purpose_surface_position, method="iqr")

# 显示筛选前后的井点数量
print(f"筛选前井点数量: {len(data_well_purpose_surface_position)}")
print(f"筛选后井点数量: {len(data_well_purpose_surface_filtered)}")

# 可视化筛选前后的井点分布
plt.figure(figsize=(12, 6))

# 计算坐标范围（使用所有井点的数据来确定范围）
x_min = data_well_purpose_surface_position["X"].min()
x_max = data_well_purpose_surface_position["X"].max()
y_min = data_well_purpose_surface_position["Y"].min()
y_max = data_well_purpose_surface_position["Y"].max()

# 可选：添加一些边距使图更美观
margin = 0.05  # 5%的边距
x_range = x_max - x_min
y_range = y_max - y_min
x_min -= x_range * margin
x_max += x_range * margin
y_min -= y_range * margin
y_max += y_range * margin

# 绘制筛选前的井点分布
plt.subplot(1, 2, 1)
plt.scatter(data_well_purpose_surface_position["X"], data_well_purpose_surface_position["Y"], c="blue")
plt.title("筛选前井点分布")
plt.xlabel("X坐标")
plt.ylabel("Y坐标")
plt.xlim(x_min, x_max)
plt.ylim(y_min, y_max)

# 绘制筛选后的井点分布
plt.subplot(1, 2, 2)
plt.scatter(data_well_purpose_surface_filtered["X"], data_well_purpose_surface_filtered["Y"], c="red")
plt.title("筛选后井点分布")
plt.xlabel("X坐标")
plt.ylabel("Y坐标")
plt.xlim(x_min, x_max)
plt.ylim(y_min, y_max)

plt.tight_layout()
plt.savefig(os.path.join(output_dir, "well_filtering_comparison.png"))
plt.show()

## 处理属性缺失值


In [ ]:
# 首先获取地震属性列表
attribute_names, _ = identify_attributes(os.path.join(data_dir, "H6-2_attr"))

# 使用preprocess_features处理地震数据
processed_seismic, attr_stats = preprocess_features(
    data=data_seismic_attr,
    attribute_columns=attribute_names,
    missing_values=[-999],
    missing_threshold=0.6,  # 缺失值超过60%的列将被删除
    outlier_method="iqr",
    outlier_threshold=1.5,
    verbose=True,
)

# 提取筛选后的属性
attribute_names_filtered = [col for col in processed_seismic.columns]

# 将处理后的属性数据与原始坐标数据合并
processed_seismic_full = data_seismic_attr[["X", "Y"]].copy()
for col in processed_seismic.columns:
    processed_seismic_full[col] = processed_seismic[col]

## 根据井点分布，缩小工区范围


In [ ]:
# 限制工区范围
seismic_attr_filtered, area_bounds = filter_seismic_by_wells(
    seismic_data=processed_seismic_full,
    well_data=data_well_purpose_surface_filtered,
    expansion_factor=1.5,  # 扩展50%
    plot=True,
    output_dir=output_dir,
)

# 后续可以直接使用area_bounds中的边界信息
print("区域边界信息:")
for key, value in area_bounds.items():
    print(f"  {key}: {value}")

## 提取井点处地震属性


In [ ]:
# 为筛选前的井点提取地震属性
well_attr = extract_seismic_attributes_for_wells(
    well_data=data_well_purpose_surface_position,
    seismic_data=processed_seismic_full,
    max_distance=50,
    num_points=5,
)

# 为筛选后的井点提取地震属性
well_attr_filtered = extract_seismic_attributes_for_wells(
    well_data=data_well_purpose_surface_filtered, seismic_data=processed_seismic_full, max_distance=50, num_points=5
)

# 保存处理结果
well_attr.to_excel(os.path.join(data_dir, "wells_attr.xlsx"), index=False)
print("筛选前井点的地震属性已保存到 wells_attr.xlsx")
well_attr_filtered.to_excel(os.path.join(data_dir, "wells_attr_filtered.xlsx"), index=False)
print("筛选后井点的地震属性已保存到 wells_attr_filtered.xlsx")

## 生成统计摘要


In [ ]:
# 筛选出质量良好的属性
good_attributes, anomalous_attributes, attribute_stats = filter_anomalous_attributes(
    seismic_data=seismic_attr_filtered,
    well_data=well_attr_filtered,
    common_attributes=attribute_names_filtered,
    ratio_threshold=5.0,  # 均值比值阈值
    range_ratio_threshold=10.0,  # 数值范围比值阈值
    std_ratio_threshold=10.0,  # 标准差比值阈值
    output_dir=None,  # 输出图表目录
    verbose=True,  # 打印详细信息
)

print("\n筛选后保留的质量良好属性:")
for attr in good_attributes:
    print(f"- {attr}")

## PCA 降维


In [ ]:
pca_results = perform_pca_analysis(
    data=seismic_attr_filtered,
    attribute_columns=good_attributes,
    variance_threshold=0.75,
    output_dir=output_dir,
)

## GMM 聚类


In [ ]:
# 首先评估最佳聚类数
gmm_evaluation = evaluate_gmm_clusters(features_pca=pca_results["features_pca"], max_clusters=10, output_dir=output_dir)

# 使用不同的聚类数执行GMM聚类
# 根据BIC/AIC结果选择的最佳聚类数
# best_n = gmm_evaluation["best_n_components"]

In [ ]:
best_n = 3  # 聚类数量

# 1. 执行GMM聚类
gmm_results = perform_gmm_clustering(
    features=pca_results["features_pca"],
    coords=pca_results["coords_clean"],
    n_clusters=best_n,
)
gmm_results["result_df"].to_csv(os.path.join(output_dir, "gmm_best_clusters.csv"), index=False)

# 2. PCA可视化，需要将井点数据投影到PCA空间
# 首先提取井点的属性列
well_features = well_attr_filtered[pca_results["features_clean"].columns].values
# 使用相同的标准化器和PCA模型变换井点数据
well_features_scaled = pca_results["scaler"].transform(well_features)
well_pca_features = pca_results["pca"].transform(well_features_scaled)

# 3. 在PCA空间中可视化聚类结果
visualize_pca_clustering(
    clustering_results=gmm_results,
    pca_results=pca_results,
    n_clusters=best_n,
    output_dir=output_dir,
    prefix="pca",
    well_data=data_well_purpose_surface_filtered,
    well_pca_features=well_pca_features,
    target_column="Sand Thickness",
    class_thresholds=[0.1, 10],
)

# 4. 在地理空间中可视化聚类结果
visualize_gmm_clustering(
    clustering_results=gmm_results,
    output_dir=output_dir,
    prefix="pca",
    well_data=data_well_purpose_surface_filtered,
    target_column="Sand Thickness",
    class_thresholds=[0.1, 10],
    point_size=120,
    well_size=200,
)